# BERTopic 토픽 모델링

In [ ]:
!pip install bertopic
!pip install sentence-transformers
!pip install konlpy


## 1. 기사 데이터 불러오기

In [ ]:
import pandas as pd

# Google Drive 마운트 (필요시)
from google.colab import drive
drive.mount('/content/drive')

# 엑셀 파일 경로 지정
df = pd.read_excel("/content/drive/MyDrive/중앙일보_토픽결과.xlsx")
docs = df["본문"].astype(str).tolist()
df.shape


## 2. 임베딩 모델 로딩 (한국어 SBERT 계열)

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")


## 3. BERTopic 학습 및 토픽 예측

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model=embedding_model, language="multilingual")
topics, probs = topic_model.fit_transform(docs)


## 4. 토픽 키워드 추출 및 기사에 붙이기

In [ ]:
topic_info = topic_model.get_topic_info()
topic_keywords = {
    row["Topic"]: ", ".join([word for word, _ in topic_model.get_topic(row["Topic"])])
    for _, row in topic_info.iterrows() if row["Topic"] != -1
}

df["토픽 번호"] = topics
df["토픽 키워드"] = df["토픽 번호"].map(topic_keywords).fillna("할당되지 않음")
df[["본문", "토픽 번호", "토픽 키워드"]].head()


## 5. 결과 저장

In [ ]:
output_path = "/content/drive/MyDrive/중앙일보_최종토픽.xlsx"
df.to_excel(output_path, index=False)
print("✅ 저장 완료:", output_path)
